In [36]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
import warnings
import sklearn

In [37]:
warnings.filterwarnings("ignore")

In [38]:
MLPRegressors= sklearn.neural_network.MLPRegressor

In [39]:
def evaluate_individual(ind):
    # Decodeer hyperparameters
    hidden_layer_sizes = tuple(int(max(1, round(x))) for x in ind[0:6])  # Zorg ervoor dat er minstens één neuron is
    activation = ['identity', 'logistic', 'tanh', 'relu'][int(ind[4]) % 4]
    alpha = 10 ** ind[4]
    learning_rate = ['constant', 'invscaling', 'adaptive'][int(ind[4])]
    learning_rate_init = 10 ** ind[3]
    power_t = ind[10]  # Geef het een geschikt bereik of conversie als dat nodig is
    max_iter = int(ind[11])  # Zorg ervoor dat dit een geheel getal is
    shuffle = bool(int(ind[12]))  # Converteer naar een bool-waarde (0 of 1)
    random_state = int(ind[13]) if ind[13] >= 0 else None  # Zorg ervoor dat het een positief getal is of None
    tol = 10 ** ind[14]
    verbose = bool(int(ind[15]))  # Converteer naar een bool-waarde (0 of 1)
    warm_start = bool(int(ind[16]))  # Converteer naar een bool-waarde (0 of 1)
    momentum = ind[17]  # Geef het een geschikt bereik of conversie als dat nodig is
    nesterovs_momentum = bool(int(ind[18]))  # Converteer naar een bool-waarde (0 of 1)
    early_stopping = bool(int(ind[19]))  # Converteer naar een bool-waarde (0 of 1)
    validation_fraction = ind[20]  # Geef het een geschikt bereik of conversie als dat nodig is
    beta_1 = (ind[21] + 1.0) / 2.0  # Geef het een geschikt bereik of conversie als dat nodig is
    beta_2 = (ind[22] + 1.0) / 2.0  # Geef het een geschikt bereik of conversie als dat nodig is
    epsilon = 10 ** ind[23]
    n_iter_no_change = int(ind[24])  # Zorg ervoor dat dit een geheel getal is
    max_fun = max(1, int(ind[4]))  # Zorg ervoor dat dit een geheel getal is  

    # Controleer of de hyperparameters binnen de gewenste bereiken liggen
    if not (0.0 <= alpha <= 1.0 and 0.0 <= power_t <= 1.0 and max_iter > 0 and tol > 0.0):
        return -1.0,  # Als een van de hyperparameters buiten het gewenste bereik ligt, retourneer een lage fitnesswaarde

    # Bouw en evalueer het neurale netwerk
    clf = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation,
                        alpha=alpha, learning_rate=learning_rate, learning_rate_init=learning_rate_init,
                        power_t=power_t, max_iter=max_iter, shuffle=shuffle, random_state=random_state,
                        tol=tol, verbose=verbose, warm_start=warm_start, momentum=momentum,
                        nesterovs_momentum=nesterovs_momentum, early_stopping=early_stopping,
                        validation_fraction=validation_fraction, beta_1=beta_1, beta_2=beta_2,
                        epsilon=epsilon, n_iter_no_change=n_iter_no_change, max_fun=max_fun)

    # Gebruik kruisvalidatie om de prestaties te beoordelen
    scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    return np.mean(scores),

In [40]:
# DEAP initialisatie
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)
toolbox = base.Toolbox()

toolbox.register("attr_float", random.uniform, -3, 3)  # Hyperparameters bounds
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=5)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate_individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

if __name__ == '__main__':
    # Laad gegevens
    data = load_iris()
    X, y = data.data, data.target
# Definieer parameters voor het genetisch algoritme
    pop_size = 10
    ngen = 10
    cxpb = 0.7
    mutpb = 0.2

    # Creëer de populatie
    population = toolbox.population(n=pop_size)

    # Draai het genetisch algoritme
    algorithms.eaMuPlusLambda(population, toolbox, mu=pop_size, lambda_=pop_size, cxpb=cxpb, mutpb=mutpb, ngen=ngen, stats=None, halloffame=None)

    # Toon het beste individu
    best_ind = tools.selBest(population, k=1)[0]
    print("Best individual:", best_ind)
    print("Best hyperparameters:", [int(best_ind[0]), ['identity', 'logistic', 'tanh', 'relu'][int(best_ind[1])], 10 ** best_ind[2], ['constant', 'invscaling', 'adaptive'][int(best_ind[3])], 10 ** best_ind[4]])

IndexError: index 10 is out of bounds for axis 0 with size 5